## Pandas Indexing Overview <a id='toc' >

Proper use of pandas indexing makes life much easier and more importantly much faster, however this is often a misunderstood capability. Wes McKinney spent a good deal of time and effort optimizing pandas through the use of indexing. In this tutorial, we will review the following:

* [Data prep](#prep)
* [Initial data exploration](#explore)
* [Basic pandas indexing](#basic)
* [Multi indexing](#multi)
* [Review how much faster indexing is](#fast)
* [Operating on groups using indexing](#groups)

In [1]:
import pandas as pd
import requests
import zipfile
import io

In [2]:
# one dataset that is really interesting is the FDA adverse events dataset 
# this can be downloaded straight from Kaggle: https://www.kaggle.com/fda/adverse-food-events. 
# I have loaded this directly in the GitHub though. 

df = pd.read_csv('../datasets/CAERS_ASCII_2004_2017Q2.csv')

In [3]:
df.head()

,RA_Report #,RA_CAERS Created Date,AEC_Event Start Date,PRI_Product Role,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Age Unit,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
0,65325,1/1/2004,8/4/2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL..."
1,65325,1/1/2004,8/4/2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL..."
2,65333,1/1/2004,NaN,Suspect,KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...,13,Ice Cream Prod,NaN,Not Available,Female,VISITED AN ER,"NAUSEA, DYSGEUSIA, DIARRHOEA"
3,65335,1/1/2004,11/24/2003,Suspect,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Prod,3.0,Month(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,"GASTROINTESTINAL DISORDER, VOMITING"
4,65336,1/1/2004,NaN,Suspect,ENFIMIL LIPIL BABY FORMULA,40,Baby Food Prod,NaN,Not Available,Not Available,VISITED A HEALTH CARE PROVIDER,"GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION"


In [4]:
print("""Dataset shape: {}
        \nDatasets type: {}""".format(df.shape,
                                     df.dtypes))

Dataset shape: (90786, 12)
        
Datasets type: RA_Report #                          int64
RA_CAERS Created Date               object
AEC_Event Start Date                object
PRI_Product Role                    object
PRI_Reported Brand/Product Name     object
PRI_FDA Industry Code                int64
PRI_FDA Industry Name               object
CI_Age at Adverse Event            float64
CI_Age Unit                         object
CI_Gender                           object
AEC_One Row Outcomes                object
SYM_One Row Coded Symptoms          object
dtype: object


### Data prep <a id='prep' >
Initial preparation of the data

* renaming columns
* converting dtypes
* normalizing age

[return to table of contents](#toc)

In [5]:
# lets prep the data just a bit

# convert CI_Age at Adverse Event to number 
# convert AEC_Event Start Date to datetime and rename to EventDate
# convert RA_CAERS Created Date to datetime and rename to ReportDate
# normalize CI_Age based on units

df.rename(columns={'RA_CAERS Created Date': 'ReportDate',
                  'AEC_Event Start Date': 'EventDate',
                  'AEC_One Row Outcomes': 'Outcome',
                  'SYM_One Row Coded Symptoms': 'Symptoms',
                  'CI_Age at Adverse Event': 'Age',
                  'CI_Gender': 'Gender',
                  'PRI_Reported Brand/Product Name': 'ProductBrandName',
                  'PRI_FDA Industry Name': 'Industry'}, 
         inplace=True)

# convert age to numeric
df['Age'] = pd.to_numeric(df['Age'], errors='coerce') # convert to numeric and convert missing
df['ReportDate'] = pd.to_datetime(df['ReportDate'],
                                 format='%m/%d/%Y')
df['EventDate'] = pd.to_datetime(df['EventDate'],
                                format='%m/%d/%Y')

In [6]:
# now we need to norm the age - lets inspect what age units we have
df['CI_Age Unit'].unique().tolist()

['Year(s)', 'Not Available', 'Month(s)', 'Week(s)', 'Day(s)', 'Decade(s)']

In [7]:
# lets create a utility func to help with the conversions

def convert_age(row):
    """
    normalize the age value based on the units. 
    i.e. if units in months, convert to years by dividing by 12
    
    :param row: pd.DataFrame row
    :return float - normalized age
    :rtype: float
    """
    age = row['Age']
    units = row['CI_Age Unit']
    
    # create lookup map
    lookup = {'Year(s)': 1.0,
             'Month(s)': 12.0,
             'Week(s)': 52.0,
             'Day(s)': 365.0,
             'Decade(s)': .1}
    
    if units != 'Not Available':
        # return the age as years
        return float(age)/lookup[units]
    
df['AgeNormed'] = df.apply(lambda row: convert_age(row), axis=1)

In [8]:
print("""Average victim age: {}
        \nYoungest victim age: {} years,
        \nOldest victim age: {} years""".format(df['AgeNormed'].mean(),
                                               df['AgeNormed'].min(),
                                               df['AgeNormed'].max()))

Average victim age: 50.39809406671898
        
Youngest victim age: 0.0 years,
        
Oldest victim age: 760.0 years


In [9]:
# there might be some data quality issues here - lets examine rows with an age 
# greater than 100 years
df[df['AgeNormed'] > 100][['Age', 'CI_Age Unit', 'AgeNormed']]

,Age,CI_Age Unit,AgeNormed
2209,76.0,Decade(s),760.0
2519,101.0,Year(s),101.0
12946,112.0,Year(s),112.0
16053,250.0,Year(s),250.0
43311,103.0,Year(s),103.0
51287,102.0,Year(s),102.0
51826,155.0,Year(s),155.0
55058,151.0,Year(s),151.0
55419,101.0,Year(s),101.0
59307,167.0,Year(s),167.0


In [10]:
# lets drop rows where AgeNormed is greater than 105 - I have never heard of someone
# surviving 76 decades - if thats true, I need to get on their diet ASAP
df = df.drop(df[df['AgeNormed'] > 105].index)

print("""Average victim age: {}
        \nYoungest victim age: {} years,
        \nOldest victim age: {} years""".format(df['AgeNormed'].mean(),
                                               df['AgeNormed'].min(),
                                               df['AgeNormed'].max()))

Average victim age: 50.368239667176425
        
Youngest victim age: 0.0 years,
        
Oldest victim age: 104.0 years


In [11]:
df[df['AgeNormed'] < 0.01] # as for the other end of hte spectrum, it does look like when 
# an age is at 0, it's due to premature babies according to the symptoms - lets leave these

,RA_Report #,ReportDate,EventDate,PRI_Product Role,ProductBrandName,PRI_FDA Industry Code,Industry,Age,CI_Age Unit,Gender,Outcome,Symptoms,AgeNormed
2531,72306,2004-09-21,2004-08-25,Suspect,ENFAMIL WITH LIPIL POWDER,40,Baby Food Prod,1.0,Day(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,DIARRHOEA,0.002740
2757,73028,2004-10-18,2004-09-20,Suspect,ENFAMIL LIPIL LOW IRON,40,Baby Food Prod,3.0,Day(s),Female,NON-SERIOUS INJURIES/ ILLNESS,"CONSTIPATION, IRRITABILITY, DYSPEPSIA, ABDOMIN...",0.008219
2758,73028,2004-10-18,2004-09-20,Suspect,ENFAMIL LIPIL WITH IRON,40,Baby Food Prod,3.0,Day(s),Female,NON-SERIOUS INJURIES/ ILLNESS,"CONSTIPATION, IRRITABILITY, DYSPEPSIA, ABDOMIN...",0.008219
6999,86973,2006-07-19,2006-06-27,Suspect,SAV-ON OSCO BY ALBERTSONS CREAMY PETROLEUM JELLY,53,Cosmetics,3.0,Day(s),Male,"VISITED A HEALTH CARE PROVIDER, DISABILITY","SKIN IRRITATION, ERYTHEMA, SKIN SWELLING",0.008219
11665,100507,2008-02-20,1996-06-18,Suspect,FLINTSTONES COMPLETE VITAMINS,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Male,"OTHER SERIOUS (IMPORTANT MEDICAL EVENTS), SERI...","FACE PRESENTATION, FOETAL MALPOSITION",0.000000
11697,100531,2008-02-21,2002-03-22,Suspect,FLINTSTONES COMPLETE VITAMINS,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Male,"HOSPITALIZATION, OTHER SERIOUS (IMPORTANT MEDI...","LUNG DISORDER, PREMATURE BABY, MULTIPLE ALLERG...",0.000000
12492,102195,2008-04-10,2007-10-31,Suspect,FLINTSTONES COMPLETE MULTIPLE VITAMINS,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Female,"HOSPITALIZATION, OTHER SERIOUS (IMPORTANT MEDI...",PREMATURE BABY,0.000000
13646,104900,2008-07-22,2005-08-22,Suspect,FLINTSTONES PLUS IRON,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Male,"HOSPITALIZATION, OTHER SERIOUS (IMPORTANT MEDI...","SMALL FOR DATES BABY, FOOD ALLERGY, GASTROOESO...",0.000000
13650,104904,2008-07-22,1995-02-25,Suspect,FLINSTONES PLUS IRON,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Female,"OTHER SERIOUS (IMPORTANT MEDICAL EVENTS), SERI...","PREMATURE BABY, SMALL FOR DATES BABY",0.000000
15916,110049,2009-02-10,NaT,Suspect,FLINTSTONES GUMMIES + IMMUNITY,54,Vit/Min/Prot/Unconv Diet(Human/Animal),0.0,Day(s),Not Available,"SERIOUS INJURIES/ ILLNESS, DEATH",STILLBIRTH,0.000000


In [12]:
# and finally, lets create some age bins
def agebins(age):
    
    if age < 2:
        return 'baby'
    
    if age >=2 and age < 13:
        return 'child'
    
    if age >= 13 and age < 20:
        return 'teenager'
    
    if age >= 20 and age < 35:
        return 'youngAdult'
    
    if age >=35 and age < 60:
        return 'adult'
    
    if age >= 60:
        return 'senior'

df['AgeBins'] = df['Age'].apply(lambda x: agebins(x))

### Initial data exploration <a id='explore' >
Let's get a better sense for what this data is about, and how we might use this to inform proper indexing levels

[return to table of contents](#toc)

In [13]:
import plotly
from plotly.offline import init_notebook_mode, plot, iplot
import cufflinks
import plotly.graph_objs as go

# initialize jupyter notebook mode
init_notebook_mode(connected=True)

plotly.__version__ # make sure we have the right version of plotly

'2.5.0'

In [14]:
# lets examine the top 10 industries with the most issues
bar_data = (df.groupby('Industry')['Age'].count()
             .reset_index()
             .rename(columns={'Age': 'NumRecords'})
             .sort_values('NumRecords', ascending=False)
             .nlargest(10, 'NumRecords'))

data = [go.Bar(
            x=bar_data['Industry'].values.tolist(),
            y=bar_data['NumRecords'].values.tolist())]

layout = go.Layout(
            title='Events by Industry',
            xaxis=dict(title='Industry', titlefont=dict(family='Courier New, monospace',
                                                       size=18)),
            yaxis=dict(title='# of Records', titlefont=dict(family='Courier New, monospace',
                                                       size=18))
)

figure = go.Figure(data=data, layout=layout)
iplot(figure)

In [15]:
# lets check out events by age

bar_data = (df.groupby('AgeBins')['Age'].count()
             .reset_index()
             .rename(columns={'Age': 'NumRecords',
                             'AgeBins': 'Age'})
             .sort_values('NumRecords', ascending=False)
             .nlargest(10, 'NumRecords'))

data = [go.Bar(
            x=bar_data['Age'].values.tolist(),
            y=bar_data['NumRecords'].values.tolist())]

layout = go.Layout(
            title='Events by Age',
            xaxis=dict(title='Age', titlefont=dict(family='Courier New, monospace',
                                                       size=18)),
            yaxis=dict(title='# of Records', titlefont=dict(family='Courier New, monospace',
                                                       size=18))
)

figure = go.Figure(data=data, layout=layout)
iplot(figure)

In [16]:
# lets also examine events by year

# remove null event date rows
years = df[df['EventDate'].notnull()].copy(deep=True)

# extract year and convert to integer
years['EventYear'] = pd.DatetimeIndex(years['EventDate']).year.astype(int).values.tolist()

yeardata = (years.groupby('EventYear')['Age']
           .count()
           .reset_index()
           .rename(columns={'EventYear': 'Year',
                                'Age': 'NumRecords'}))

# filter to last 20 years - drop 2017 due to incomplete collection
yeardata = yeardata[(yeardata['Year'] >= 2000) & (yeardata['Year'] < 2017)]

data = [go.Scatter(
            x=yeardata['Year'].values.tolist(),
            y=yeardata['NumRecords'].values.tolist()
)]

layout = go.Layout(
            title='Events by Year',
            xaxis=dict(title='Year', titlefont=dict(family='Courier New, monospace',
                                                       size=18)),
            yaxis=dict(title='# of Records', titlefont=dict(family='Courier New, monospace',
                                                       size=18))
)

figure = go.Figure(data=data, layout=layout)
iplot(figure)

### Basic pandas indexing <a id='basic' >
* How to set an index
* How to retrieve via index

[return to table of contents](#toc)

In [17]:
# setting an index in pandas is quite easy - lets use AgeBins to start

hdf = df.set_index('AgeBins')

# lets select all rows that are related to children
hdf.loc['child', :].head()

,RA_Report #,ReportDate,EventDate,PRI_Product Role,ProductBrandName,PRI_FDA Industry Code,Industry,Age,CI_Age Unit,Gender,Outcome,Symptoms,AgeNormed
AgeBins,,,,,,,,,,,,,
child,65325,2004-01-01,2003-08-04,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL...",2.00
child,65325,2004-01-01,2003-08-04,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL...",2.00
child,65335,2004-01-01,2003-11-24,Suspect,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Prod,3.0,Month(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,"GASTROINTESTINAL DISORDER, VOMITING",0.25
child,65345,2004-01-01,2003-12-21,Suspect,"FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS",7,Snack Food Item,10.0,Year(s),Male,NON-SERIOUS INJURIES/ ILLNESS,CHOKING,10.00
child,65439,2004-01-06,2003-12-15,Suspect,DORMANS SWISS CHEESE,12,Cheese/Cheese Prod,4.0,Year(s),Female,NON-SERIOUS INJURIES/ ILLNESS,MALAISE,4.00


In [18]:
# you can create a boolean mask to pass from analysis to analysis as well
cmask = hdf.index=='child'
# select age for all children
hdf.loc[cmask, 'Age'].head()

AgeBins
child     2.0
child     2.0
child     3.0
child    10.0
child     4.0
Name: Age, dtype: float64

### Multi indexing <a id='multi' >
pandas supports creating multiple levels within your indexing. You can construct the 
indices in several different ways

[return to table of contents](#toc)

In [19]:
hdf = df.set_index(['AgeBins', 'Industry'])
hdf.head()

RA_Report # ReportDate  EventDate  \
AgeBins Industry                                                         
child   Bakery Prod/Dough/Mix/Icing        65325 2004-01-01 2003-08-04   
        Bakery Prod/Dough/Mix/Icing        65325 2004-01-01 2003-08-04   
NaN     Ice Cream Prod                     65333 2004-01-01        NaT   
child   Baby Food Prod                     65335 2004-01-01 2003-11-24   
NaN     Baby Food Prod                     65336 2004-01-01        NaT   

                                    PRI_Product Role  \
AgeBins Industry                                       
child   Bakery Prod/Dough/Mix/Icing          Suspect   
        Bakery Prod/Dough/Mix/Icing          Suspect   
NaN     Ice Cream Prod                       Suspect   
child   Baby Food Prod                       Suspect   
NaN     Baby Food Prod                       Suspect   

                                                                      ProductBrandName  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing      MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS   
        Bakery Prod/Dough/Mix/Icing      MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS   
NaN     Ice Cream Prod               KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...   
child   Baby Food Prod                                      ENFAMIL LIPIL BABY FORMULA   
NaN     Baby Food Prod                                      ENFIMIL LIPIL BABY FORMULA   

                                     PRI_FDA Industry Code  Age  \
AgeBins Industry                                                  
child   Bakery Prod/Dough/Mix/Icing                      3  2.0   
        Bakery Prod/Dough/Mix/Icing                      3  2.0   
NaN     Ice Cream Prod                                  13  NaN   
child   Baby Food Prod                                  40  3.0   
NaN     Baby Food Prod                                  40  NaN   

                                       CI_Age Unit         Gender  \
AgeBins Industry                                                    
child   Bakery Prod/Dough/Mix/Icing        Year(s)         Female   
        Bakery Prod/Dough/Mix/Icing        Year(s)         Female   
NaN     Ice Cream Prod               Not Available         Female   
child   Baby Food Prod                    Month(s)  Not Available   
NaN     Baby Food Prod               Not Available  Not Available   

                                                                               Outcome  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing  VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...   
        Bakery Prod/Dough/Mix/Icing  VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...   
NaN     Ice Cream Prod                                                   VISITED AN ER   
child   Baby Food Prod                                   NON-SERIOUS INJURIES/ ILLNESS   
NaN     Baby Food Prod                                  VISITED A HEALTH CARE PROVIDER   

                                                                              Symptoms  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing  SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL...   
        Bakery Prod/Dough/Mix/Icing  SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL...   
NaN     Ice Cream Prod                                    NAUSEA, DYSGEUSIA, DIARRHOEA   
child   Baby Food Prod                             GASTROINTESTINAL DISORDER, VOMITING   
NaN     Baby Food Prod                 GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION   

                                     AgeNormed  
AgeBins Industry                                
child   Bakery Prod/Dough/Mix/Icing       2.00  
        Bakery Prod/Dough/Mix/Icing       2.00  
NaN     Ice Cream Prod                     NaN  
child   Baby Food Prod           

In [20]:
# lets create multiple indices - agebins and industry
hdf = df.set_index(['AgeBins', 'Industry'])

# we can select all adults that had an event related to cosmetics with the 
# following:
hdf.loc[('adult', 'Cosmetics'), :].head()

/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: PerformanceWarning:

indexing past lexsort depth may impact performance.



RA_Report # ReportDate EventDate PRI_Product Role  \
AgeBins Industry                                                       
adult   Cosmetics        65973 2004-01-22       NaT          Suspect   
        Cosmetics        65973 2004-01-22       NaT          Suspect   
        Cosmetics        66135 2004-01-28       NaT          Suspect   
        Cosmetics        66201 2004-01-29       NaT          Suspect   
        Cosmetics        66201 2004-01-29       NaT          Suspect   

                                                    ProductBrandName  \
AgeBins Industry                                                       
adult   Cosmetics  PREMIER PIGMENTS PERMANENT TATTOO PIGMENTS: RO...   
        Cosmetics  PREMIER PIGMENTS PERMANENT TATTOO PIGMENTS: RO...   
        Cosmetics  OPTIMUM OPTI INTENSE BODY HEAT ACTIVATED CONDI...   
        Cosmetics                       FLAGEOLL MAGNOLIA BATH FLOAT   
        Cosmetics                                 MAGNOLIA BODY MIST   

                   PRI_FDA Industry Code   Age CI_Age Unit  Gender  \
AgeBins Industry                                                     
adult   Cosmetics                     53  51.0     Year(s)  Female   
        Cosmetics                     53  51.0     Year(s)  Female   
        Cosmetics                     53  42.0     Year(s)  Female   
        Cosmetics                     53  52.0     Year(s)  Female   
        Cosmetics                     53  52.0     Year(s)  Female   

                                                         Outcome  \
AgeBins Industry                                                   
adult   Cosmetics                  NON-SERIOUS INJURIES/ ILLNESS   
        Cosmetics                  NON-SERIOUS INJURIES/ ILLNESS   
        Cosmetics  VISITED A HEALTH CARE PROVIDER, VISITED AN ER   
        Cosmetics                  NON-SERIOUS INJURIES/ ILLNESS   
        Cosmetics                  NON-SERIOUS INJURIES/ ILLNESS   

                                                            Symptoms  \
AgeBins Industry                                                       
adult   Cosmetics  BLISTER, SWELLING FACE, CHEILITIS, PRURITUS, H...   
        Cosmetics  SWELLING FACE, CHEILITIS, BLISTER, PRURITUS, H...   
        Cosmetics  EMERGENCY CARE EXAMINATION, CONJUNCTIVAL HYPER...   
        Cosmetics   BLISTER, SCAR, CAUSTIC INJURY, BURNING SENSATION   
        Cosmetics   BLISTER, SCAR, CAUSTIC INJURY, BURNING SENSATION   

                   AgeNormed  
AgeBins Industry              
adult   Cosmetics       51.0  
        Cosmetics       51.0  
        Cosmetics       42.0  
        Cosmetics       52.0  
        Cosmetics       52.0

In [21]:
# however, you may notice that the indices are not sorted
hdf.head()

RA_Report # ReportDate  EventDate  \
AgeBins Industry                                                         
child   Bakery Prod/Dough/Mix/Icing        65325 2004-01-01 2003-08-04   
        Bakery Prod/Dough/Mix/Icing        65325 2004-01-01 2003-08-04   
NaN     Ice Cream Prod                     65333 2004-01-01        NaT   
child   Baby Food Prod                     65335 2004-01-01 2003-11-24   
NaN     Baby Food Prod                     65336 2004-01-01        NaT   

                                    PRI_Product Role  \
AgeBins Industry                                       
child   Bakery Prod/Dough/Mix/Icing          Suspect   
        Bakery Prod/Dough/Mix/Icing          Suspect   
NaN     Ice Cream Prod                       Suspect   
child   Baby Food Prod                       Suspect   
NaN     Baby Food Prod                       Suspect   

                                                                      ProductBrandName  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing      MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS   
        Bakery Prod/Dough/Mix/Icing      MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS   
NaN     Ice Cream Prod               KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...   
child   Baby Food Prod                                      ENFAMIL LIPIL BABY FORMULA   
NaN     Baby Food Prod                                      ENFIMIL LIPIL BABY FORMULA   

                                     PRI_FDA Industry Code  Age  \
AgeBins Industry                                                  
child   Bakery Prod/Dough/Mix/Icing                      3  2.0   
        Bakery Prod/Dough/Mix/Icing                      3  2.0   
NaN     Ice Cream Prod                                  13  NaN   
child   Baby Food Prod                                  40  3.0   
NaN     Baby Food Prod                                  40  NaN   

                                       CI_Age Unit         Gender  \
AgeBins Industry                                                    
child   Bakery Prod/Dough/Mix/Icing        Year(s)         Female   
        Bakery Prod/Dough/Mix/Icing        Year(s)         Female   
NaN     Ice Cream Prod               Not Available         Female   
child   Baby Food Prod                    Month(s)  Not Available   
NaN     Baby Food Prod               Not Available  Not Available   

                                                                               Outcome  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing  VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...   
        Bakery Prod/Dough/Mix/Icing  VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...   
NaN     Ice Cream Prod                                                   VISITED AN ER   
child   Baby Food Prod                                   NON-SERIOUS INJURIES/ ILLNESS   
NaN     Baby Food Prod                                  VISITED A HEALTH CARE PROVIDER   

                                                                              Symptoms  \
AgeBins Industry                                                                         
child   Bakery Prod/Dough/Mix/Icing  SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL...   
        Bakery Prod/Dough/Mix/Icing  SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL...   
NaN     Ice Cream Prod                                    NAUSEA, DYSGEUSIA, DIARRHOEA   
child   Baby Food Prod                             GASTROINTESTINAL DISORDER, VOMITING   
NaN     Baby Food Prod                 GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION   

                                     AgeNormed  
AgeBins Industry                                
child   Bakery Prod/Dough/Mix/Icing       2.00  
        Bakery Prod/Dough/Mix/Icing       2.00  
NaN     Ice Cream Prod                     NaN  
child   Baby Food Prod           

In [22]:
# to sort our indices lexicographically (alphabetically) (which pandas will look for in more complex queries)
# we can use:

hdf.sort_index(inplace=True,
              level=[0, 1])

# check that both levels were sorted
hdf.index.lexsort_depth # both levels are sorted against

2

In [23]:
# now we can select ranges of items - lets get all records from adult to child 
# without explicitly defining each level (adult, baby, child) - remember its alphabetically
# sorted

hdf.loc[slice('adult', 'child'), :].head()

RA_Report # ReportDate  EventDate  \
AgeBins Industry                                                
adult   Alcoholic Beverage        74823 2004-12-23 2004-12-14   
        Alcoholic Beverage        75482 2005-01-20        NaT   
        Alcoholic Beverage        77994 2005-05-10 2005-05-05   
        Alcoholic Beverage        78005 2005-05-11 2005-05-05   
        Alcoholic Beverage        92110 2007-03-19 2007-02-28   

                           PRI_Product Role     ProductBrandName  \
AgeBins Industry                                                   
adult   Alcoholic Beverage      Concomitant      BOURBON WHISKEY   
        Alcoholic Beverage          Suspect  PRINCESA MALT DRINK   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage      Concomitant           WINE SAUCE   

                            PRI_FDA Industry Code   Age CI_Age Unit  Gender  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                     32  37.0     Year(s)    Male   
        Alcoholic Beverage                     32  49.0     Year(s)  Female   
        Alcoholic Beverage                     32  42.0     Year(s)  Female   
        Alcoholic Beverage                     32  40.0     Year(s)  Female   
        Alcoholic Beverage                     32  38.0     Year(s)    Male   

                                                                     Outcome  \
AgeBins Industry                                                               
adult   Alcoholic Beverage             DEATH, HOSPITALIZATION, VISITED AN ER   
        Alcoholic Beverage                                     VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage  VISITED AN ER, LIFE THREATENING, HOSPITALIZATION   

                                                                   Symptoms  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                                            DEATH   
        Alcoholic Beverage                          ABDOMINAL PAIN, MALAISE   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage  DIARRHOEA, MALAISE, DEHYDRATION, ABDOMINAL PAIN   

                            AgeNormed  
AgeBins Industry                       
adult   Alcoholic Beverage       37.0  
        Alcoholic Beverage       49.0  
        Alcoholic Beverage       42.0  
        Alcoholic Beverage       40.0  
        Alcoholic Beverage       38.0

In [24]:
# and to select ranges for both levels - in this case all 
# rows that are adult through child, and in industry alcoholic beverage through
# choc/cocoa products

hdf.loc[(slice('adult', 'child'), slice('Alcoholic Beverage', 'Choc/Cocoa Prod')), :].head()

RA_Report # ReportDate  EventDate  \
AgeBins Industry                                                
adult   Alcoholic Beverage        74823 2004-12-23 2004-12-14   
        Alcoholic Beverage        75482 2005-01-20        NaT   
        Alcoholic Beverage        77994 2005-05-10 2005-05-05   
        Alcoholic Beverage        78005 2005-05-11 2005-05-05   
        Alcoholic Beverage        92110 2007-03-19 2007-02-28   

                           PRI_Product Role     ProductBrandName  \
AgeBins Industry                                                   
adult   Alcoholic Beverage      Concomitant      BOURBON WHISKEY   
        Alcoholic Beverage          Suspect  PRINCESA MALT DRINK   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage      Concomitant           WINE SAUCE   

                            PRI_FDA Industry Code   Age CI_Age Unit  Gender  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                     32  37.0     Year(s)    Male   
        Alcoholic Beverage                     32  49.0     Year(s)  Female   
        Alcoholic Beverage                     32  42.0     Year(s)  Female   
        Alcoholic Beverage                     32  40.0     Year(s)  Female   
        Alcoholic Beverage                     32  38.0     Year(s)    Male   

                                                                     Outcome  \
AgeBins Industry                                                               
adult   Alcoholic Beverage             DEATH, HOSPITALIZATION, VISITED AN ER   
        Alcoholic Beverage                                     VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage  VISITED AN ER, LIFE THREATENING, HOSPITALIZATION   

                                                                   Symptoms  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                                            DEATH   
        Alcoholic Beverage                          ABDOMINAL PAIN, MALAISE   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage  DIARRHOEA, MALAISE, DEHYDRATION, ABDOMINAL PAIN   

                            AgeNormed  
AgeBins Industry                       
adult   Alcoholic Beverage       37.0  
        Alcoholic Beverage       49.0  
        Alcoholic Beverage       42.0  
        Alcoholic Beverage       40.0  
        Alcoholic Beverage       38.0

In [25]:
# however the slice object isn't ideal - we can use pandas IndexSlice to perform 
# in a more compact way
idx = pd.IndexSlice

# lets perform the same query as above with our new slicer
hdf.loc[idx[['adult', 'child'], ['Alcoholic Beverage', 'Choc/Cocoa Prod']], :].head()

RA_Report # ReportDate  EventDate  \
AgeBins Industry                                                
adult   Alcoholic Beverage        74823 2004-12-23 2004-12-14   
        Alcoholic Beverage        75482 2005-01-20        NaT   
        Alcoholic Beverage        77994 2005-05-10 2005-05-05   
        Alcoholic Beverage        78005 2005-05-11 2005-05-05   
        Alcoholic Beverage        92110 2007-03-19 2007-02-28   

                           PRI_Product Role     ProductBrandName  \
AgeBins Industry                                                   
adult   Alcoholic Beverage      Concomitant      BOURBON WHISKEY   
        Alcoholic Beverage          Suspect  PRINCESA MALT DRINK   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage          Suspect        BLUE NUN WINE   
        Alcoholic Beverage      Concomitant           WINE SAUCE   

                            PRI_FDA Industry Code   Age CI_Age Unit  Gender  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                     32  37.0     Year(s)    Male   
        Alcoholic Beverage                     32  49.0     Year(s)  Female   
        Alcoholic Beverage                     32  42.0     Year(s)  Female   
        Alcoholic Beverage                     32  40.0     Year(s)  Female   
        Alcoholic Beverage                     32  38.0     Year(s)    Male   

                                                                     Outcome  \
AgeBins Industry                                                               
adult   Alcoholic Beverage             DEATH, HOSPITALIZATION, VISITED AN ER   
        Alcoholic Beverage                                     VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage      NON-SERIOUS INJURIES/ ILLNESS, VISITED AN ER   
        Alcoholic Beverage  VISITED AN ER, LIFE THREATENING, HOSPITALIZATION   

                                                                   Symptoms  \
AgeBins Industry                                                              
adult   Alcoholic Beverage                                            DEATH   
        Alcoholic Beverage                          ABDOMINAL PAIN, MALAISE   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage                                       LACERATION   
        Alcoholic Beverage  DIARRHOEA, MALAISE, DEHYDRATION, ABDOMINAL PAIN   

                            AgeNormed  
AgeBins Industry                       
adult   Alcoholic Beverage       37.0  
        Alcoholic Beverage       49.0  
        Alcoholic Beverage       42.0  
        Alcoholic Beverage       40.0  
        Alcoholic Beverage       38.0

### How much faster is indexing than traditional conditional lookups? <a id='fast' >

Lets run a quick simulation to get a sense for how much faster pandas indexing really is

[return to table of contents](#toc)

In [26]:
import timeit

# lets set up a scenario to select all adult, baby and child rows
start = timeit.default_timer()

for i in range(1000):
    z = hdf.loc[idx[['adult', 'child'], :], :]
    
elapsed = timeit.default_timer() - start
print(elapsed)

3.9972034670063294


In [27]:
start = timeit.default_timer()

# and for traditional conditional lookups
for i in range(1000):
    z = df.loc[(df['AgeBins'] == 'adult') & (df['AgeBins'] == 'baby') & (df['AgeBins'] == 'child'), :]
    
elapsed2 = timeit.default_timer() - start
print(elapsed2)

13.471444902999792


In [28]:
print("""Total time for 1000 iterations: 
        \nUsing traditional conditional selection: {}
        \nUsing pandas indexing: {}""".format(elapsed2, elapsed))

Total time for 1000 iterations: 
        
Using traditional conditional selection: 13.471444902999792
        
Using pandas indexing: 3.9972034670063294


### Operating on groups and getting summary statistics <a id='groups' >
Lets see how we can get summary stats for varying levels of an index

[return to table of contents](#toc)

In [29]:
# lets get the average age of each agebin
hdf['Age'].mean(level=0) # simple as that

AgeBins
adult         47.894728
baby           0.935484
child          6.207740
senior        72.895774
teenager      16.189895
youngAdult    27.791237
Name: Age, dtype: float64

In [30]:
# and if we want it for both levels - average age of 
# reported event victim per bin by industry
hdf['Age'].mean(level=[0, 1]).head()

AgeBins  Industry                       
adult    Alcoholic Beverage                 40.833333
         Baby Food Prod                     48.428571
         Bakery Prod/Dough/Mix/Icing        48.786730
         Beverage Bases/Conc/Nectar         45.666667
         Candy W/O Choc/Special/Chew Gum    46.661017
Name: Age, dtype: float64

In [31]:
# and just the average age for industry
(hdf['Age'].mean(level=1)
 .reset_index(name='AvgAge')
 .sort_values('AvgAge', ascending=False)
 .nsmallest(10, 'AvgAge'))

,Industry,AvgAge
1,Baby Food Prod,7.659772
26,Miscellaneous Food Related Items,21.181818
4,Candy W/O Choc/Special/Chew Gum,28.413043
32,Snack Food Item,29.480122
23,Macaroni/Noodle Prod,29.483871
2,Bakery Prod/Dough/Mix/Icing,30.197905
9,Color Additiv Food/Drug/Cosmetic,30.384615
7,Choc/Cocoa Prod,31.158798
35,"Spices, Flavors And Salts",31.750000
3,Beverage Bases/Conc/Nectar,32.525424


In [32]:
# get the diff between max and min age by agebin in the index
hdf['Age'].groupby(level=0).apply(lambda x: x.max() - x.min()).reset_index(name='diff')

,AgeBins,diff
0,adult,24.0
1,baby,1.0
2,child,10.0
3,senior,676.0
4,teenager,6.0
5,youngAdult,14.0


In [33]:
# and finally, to simply return to your normal, unindexed dataframe
# you can use reset_index()

hdf.reset_index().tail()

,AgeBins,Industry,RA_Report #,ReportDate,EventDate,PRI_Product Role,ProductBrandName,PRI_FDA Industry Code,Age,CI_Age Unit,Gender,Outcome,Symptoms,AgeNormed
90772,youngAdult,Whole Grain/Milled Grain Prod/Starch,178753,2014-08-22,2014-08-10,Suspect,BOBS RED MILL SWEET WHITE SORGHUM FLOUR,2,34.0,Year(s),Female,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),"HYPERSENSITIVITY, FATIGUE, DIZZINESS, PAIN, DY...",34.0
90773,youngAdult,Whole Grain/Milled Grain Prod/Starch,192971,2016-01-22,2016-01-02,Suspect,"POP WEAVER MICROWAVE POPCORN, EXTRA BUTTER",2,22.0,Year(s),Female,"VISITED A HEALTH CARE PROVIDER, NON-SERIOUS IN...","DIARRHOEA, NAUSEA, VOMITING",22.0
90774,youngAdult,Whole Grain/Milled Grain Prod/Starch,196083,2016-05-10,2016-04-12,Suspect,SWEET CORN,2,28.0,Year(s),Female,"NON-SERIOUS INJURIES/ ILLNESS, DISABILITY","LISTERIOSIS, DIARRHOEA, DEHYDRATION, VOMITING,...",28.0
90775,youngAdult,Whole Grain/Milled Grain Prod/Starch,204948,2016-12-07,2016-12-02,Suspect,PAMONHA,2,27.0,Year(s),Male,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),"DIARRHOEA, MALAISE",27.0
90776,youngAdult,Whole Grain/Milled Grain Prod/Starch,205605,2016-12-22,2016-12-02,Suspect,RENATO GOMES PAMONHA,2,27.0,Year(s),Male,NON-SERIOUS INJURIES/ ILLNESS,"DIARRHOEA, MALAISE",27.0
